# Reading simulation data from GEANT4 .root output files

Results of Geant4 simulations are stored in ROOT tree format in .root files with naming scheme as "PICOSpec_TKE_A1_A2.root", where TKE is a total kinetic energy of fission fragments with mass numbers A1 and A2.

# Importing python packages<a name="import"></a>

Packages for reading, storing and manipulating data:

In [1]:
import ROOT # for working with Geant4 output files of .root format 
import pandas as pd # to process large data sets
import numpy as np # to perform numerical operations

Welcome to JupyROOT 6.18/02


Packages used in order to find all .root files in specified location 

In [2]:
import os # to be able to perform pathways manipulations
import glob # for finding files of specific format 
import re # used for reading int variables from filename

# Extracting information from ROOT trees

Extracting information from ROOT trees into pandas DataFrame. 

In [3]:
# Creating DataFrame to fill from ROOT data files
data = pd.DataFrame({'TKE': [], 'A1': [], 'A2': [], 'Event': [], 'Energy': [], 'DetID': []}) 
data

,TKE,A1,A2,Event,Energy,DetID


In [4]:
#for filepath in glob.glob(os.path.join('resources', '*.root')):
for filepath in glob.glob(os.path.join('data', '*.root')):
    print('Reading data from file: ', filepath)
    regex = re.compile(r'\d+')
    idet = [int(x) for x in regex.findall(filepath)] # read TKE, A1, A2 from filename
    print('TKE={}, A1={}, A2={}'.format(idet[0], idet[1], idet[2]))
    f = ROOT.TFile.Open(filepath)
    rawTree = f.Get("PICO_tree_raw")
    for idx, event in enumerate(rawTree): 
        for i in range(event.Mult):
            data = data.append({'TKE': idet[0], 'A1': idet[1], 'A2': idet[2], 'Event': idx,'Energy': event.Energy[i], 'DetID': event.det_ID[i]}, ignore_index=True)


Reading data from file:  data/PICOSpec_275_142_142.root
TKE=275, A1=142, A2=142
Reading data from file:  data/PICOSpec_250_76_208.root
TKE=250, A1=76, A2=208
Reading data from file:  data/PICOSpec_275_76_208.root
TKE=275, A1=76, A2=208
Reading data from file:  data/PICOSpec_225_76_208.root
TKE=225, A1=76, A2=208
Reading data from file:  data/PICOSpec_275_109_175.root
TKE=275, A1=109, A2=175
Reading data from file:  data/PICOSpec_250_109_175.root
TKE=250, A1=109, A2=175
Reading data from file:  data/PICOSpec_225_142_142.root
TKE=225, A1=142, A2=142
Reading data from file:  data/PICOSpec_250_142_142.root
TKE=250, A1=142, A2=142
Reading data from file:  data/PICOSpec_225_109_175.root
TKE=225, A1=109, A2=175


In [8]:
data = data.astype({'A1':'int', 'A2':'int', 'Event':'int', 'DetID':'int'})

In [9]:
data

,TKE,A1,A2,Event,Energy,DetID
0,275.0,142,142,0,194026.558304,593
1,275.0,142,142,0,21016.307741,1209
2,275.0,142,142,1,201667.370187,878
3,275.0,142,142,1,3609.370434,1919
4,275.0,142,142,2,198400.728601,496
...,...,...,...,...,...,...
30066,225.0,109,175,2490,27263.896950,1468
30067,225.0,109,175,2491,182485.562287,494
30068,225.0,109,175,2492,197704.526139,594
30069,225.0,109,175,2493,138511.989400,558


In [10]:
data.set_index(['TKE', 'A1', 'A2'],inplace=True)

Sorting index before slicing to avoid pandas performance warning:

In [11]:
data = data.sort_index()
data.index.is_lexsorted()

True

Below is the final `DataFrame` with all relevant results produced by running simulations in Geant4. Simulations were run with fission products **A1** and **A2** and total kinetic energy **TKE**. For each simulations all **Event**s were extracted from ROOT trees. There are multiple hits of different **Energy** in different pixels **DetID** of detectors of TASISpec set-up.

In [12]:
data

Event         Energy  DetID
TKE   A1  A2                              
225.0 76  208      0  204701.274291    593
          208      1  210576.180404    433
          208      2  203917.364536    433
          208      3  225000.000000    560
          208      4   90427.887638    591
...              ...            ...    ...
275.0 142 142   2490  275000.000000    336
          142   2491  237954.660832    716
          142   2492  250077.094510    430
          142   2493  189529.601536    372
          142   2493   13364.919537   1788

[30071 rows x 3 columns]

## Storing extracted ROOT data as DataFrame into binary file

In [13]:
data.to_pickle('fission_data.pk1')

In [14]:
del data 